In [31]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# how big the data is 

In [77]:
df.shape

(20, 7)

# What is the data type of the columns

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


# finding the missing values

In [6]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [76]:
df.describe()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
count,20,20,20.0,20,20,20.0,20
unique,3,2,20.0,3,3,20.0,3
top,2,male,55.5,1,1,34.2,S
freq,7,10,1.0,8,9,1.0,14


In [8]:
#for duplicates we have a function
df.duplicated().sum()

np.int64(0)

In [9]:
df = df.drop(columns=['PassengerId', 'Ticket', 'Cabin', 'Name'], errors='ignore')
df.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C


In [10]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),
                                                 df['Survived'],
                                                 test_size=0.2,
                                                random_state=42)

### We saw that there is null values in the data so we will first fill them using SimpleImputation library

In [11]:
tr1 = ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')

In [12]:
tr2 = ColumnTransformer(transformers=[
    ('Sex_embarked', OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')

In [13]:
tr3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])

In [14]:
tr4 = DecisionTreeClassifier()

In [15]:
tr = Pipeline([('tr1',tr1),('tr2',tr2),('tr3',tr3),('tr4',tr4)])

In [16]:
tr.fit(X_train,y_train)

,steps,"[('tr1', ...), ('tr2', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('impute_age', ...), ('impute_embarked', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [18]:
tr.named_steps

{'tr1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'tr2': ColumnTransformer(remainder='passthrough',
                   transformers=[('Sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 6])]),
 'tr3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'tr4': DecisionTreeClassifier()}

In [25]:
y_pred = tr.predict(X_test)

In [26]:
y_pred

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0])

#### we can cross validate using the same algorith for multiple time

In [33]:
cross_val_score(tr, X_train, y_train, cv=60, scoring='accuracy').mean()  # cv is the repeatation time

np.float64(0.6392676767676768)

#### Will make pickle to explore this pipeline

In [41]:
import pickle as pk
pk.dump(tr,open('model.pkl','wb'))

# we will load pickle to predict few data

In [65]:
pipe = pk.load(open('model.pkl', 'rb'))

In [66]:
ndf= pd.read_csv('New_data_to_predict.csv')

In [67]:
ndf.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,2,male,55.5,1,2,34.2,S
1,1,female,23.4,2,0,10.0,C
2,3,female,39.6,1,1,20.5,S
3,2,male,24.7,2,1,16.0,S
4,3,male,47.2,0,1,23.3,S


In [69]:
x =pipe.predict(ndf)

In [75]:
y= pd.DataFrame(x,columns=['Survived'])
pd.concat([y,ndf],axis=1)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,2,male,55.5,1,2,34.2,S
1,1,1,female,23.4,2,0,10.0,C
2,0,3,female,39.6,1,1,20.5,S
3,0,2,male,24.7,2,1,16.0,S
4,0,3,male,47.2,0,1,23.3,S
5,1,2,male,58.1,1,0,19.8,C
6,0,1,female,26.1,0,1,40.7,S
7,0,2,female,43.0,2,2,14.5,S
8,0,3,female,48.9,0,1,33.5,S
9,0,1,male,37.2,2,0,24.9,Q
